In [41]:
class Operation():
    
    def __init__(self, input_nodes = []):
       # Set up input nodes to the operation          
        self.input_nodes = input_nodes
        
       # where nodes are outputted to         
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
    
    def compute(self):
        pass 
        

In [42]:
class add(Operation):
    
    def __init__(self,x,y):
        
        super.__init__([x,y])
        
    def compute(self,x_var,y_var):
        self.inputs=[x_var,y_var]
        return x_var + y_var
        
        
        
    

In [43]:
class multiply(Operation):
    
    def __init__(self,x,y):
        
        super.__init__([x,y])
        
    def compute(self,x_var,y_var):
        self.inputs=[x_var,y_var]
        return x_var * y_var
        
        
        


In [44]:
class matrix_multiply(Operation):
    
    def __init__(self,x,y):
        
        super.__init__([x,y])
        
    def compute(self,x_var,y_var):
        self.inputs=[x_var,y_var]
        return x_var.dot(y_var)
        
        
        


In [ ]:
# Placeholder - an empty node that needs a value to be provided a compute output
# Variables - Changeable paramter of a graph
# Graph - global variables connecting variables and placeholders to operations